<a href="https://colab.research.google.com/github/umeshnmenon/EVA4/blob/master/S4_Assignment_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DNN To recognize handwritten digit - using MNIST data

In [0]:
"""
This Notebook helps create a DNN using PyTorch to recognize the handwritten digit using MNIST data
"""
# import all the libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

DNN Architecture

In [0]:
# This cell contains the definition of class to create the Deep Neural Net
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # Define the architecture
        # Usage: Conv2d(in_channels, out_channels, kernel_size)
        
             
        # architecture
        self.conv1 = nn.Sequential(nn.Conv2d(1, 16, 3, bias=False),             #input = 1 Output = 16 RF = 3
                                   nn.ReLU(),                 
                                   nn.BatchNorm2d(16),
                                   nn.Dropout2d(0.05))                          # size = 26

        self.conv2 = nn.Sequential(nn.Conv2d(16, 32, 3, bias=False),            #input = 16 Output = 32 RF = 5
                                   nn.ReLU(),
                                   nn.BatchNorm2d(32),
                                   nn.Dropout2d(0.05))                          # size = 24
        
        self.conv3 = nn.Sequential(nn.Conv2d(32, 8, 1, bias=False))             #input = 32 Output = 8 RF = 10
                                                                                # size = 24

        self.maxpool = nn.MaxPool2d(2, 2)                                       # maxpooling 2x2 kernel and stride 2
                                                                                # size = 12
        #self.dropout = nn.Dropout2d(0.05)

        self.conv4 = nn.Sequential(nn.Conv2d(8, 16, 3, bias=False),             #input = 8 Output = 16 RF = 12
                                   nn.ReLU(),
                                   nn.BatchNorm2d(16),
                                   nn.Dropout2d(0.05))                          # size = 10

        self.conv5 = nn.Sequential(nn.Conv2d(16, 16, 3, bias=False),            #input = 16 Output = 16 RF = 14
                                   nn.ReLU(),
                                   nn.BatchNorm2d(16),
                                   nn.Dropout2d(0.05))                          # size = 8
        
        self.conv6 = nn.Sequential(nn.Conv2d(16, 16, 3, padding=1, bias=False), #input = 16 Output = 16 RF = 16
                                   nn.ReLU(),
                                   nn.BatchNorm2d(16),
                                   nn.Dropout2d(0.05))                          # size = 8
        
        self.conv7 = nn.Sequential(nn.Conv2d(16, 10, 3, bias=False))            #input = 16 Output = 10 RF = 18
                                                                                # size = 6

        self.GAP = nn.AvgPool2d(6)                                              # size = 1
        

    def forward(self, x):
        
        # Convolution Block 1
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        # Transformation Block 1
        x = self.maxpool(x)
        # Convolution Block 2
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.GAP(x)
        #x = x.view(x.size(0), -1)
        x = x.view(-1, 10)
       
        return F.log_softmax(x)

In [3]:
# This cell contains code initialize the model
# Install torchsummary to get the model summary
!pip install torchsummary
from torchsummary import summary
# check for GPU
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
# intialize the model
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
         Dropout2d-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 32, 24, 24]           4,608
              ReLU-6           [-1, 32, 24, 24]               0
       BatchNorm2d-7           [-1, 32, 24, 24]              64
         Dropout2d-8           [-1, 32, 24, 24]               0
            Conv2d-9            [-1, 8, 24, 24]             256
        MaxPool2d-10            [-1, 8, 12, 12]               0
           Conv2d-11           [-1, 16, 10, 10]           1,152
             ReLU-12           [-1, 16, 10, 10]               0
      BatchNorm2d-13           [-1, 16, 10, 10]              32
        Dropout2d-14           [-1, 16,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



*   Load Data
*   Split to train and test set
*   Set the batch size

In [0]:
# This cell contains code to get the data iterables for Train and Test data set


# random seed to reproduce the results
torch.manual_seed(1)
# define a batch size to select from the input sample for epoch
batch_size = 128
# arguments for GPU
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
# Loads the train and test set
# Will apply a set of transforms to the input image. First we need to convert them
# to Tensors and then Normalize them using given mean and sd
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Train and Test functions

In [0]:
# This cell contains code to train and test the model
from tqdm import tqdm # for the progress bar


def train(model, device, train_loader, optimizer, epoch):
    """
    Trains a given model using the train data
    """
    # set model to training mode
    model.train()
    pbar = tqdm(train_loader)
    # loop through the data
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        # reset gradients to zero
        optimizer.zero_grad()
        # do a forward pass
        output = model(data)
        # calculate the loss. Our loss is negative log likelihood
        loss = F.nll_loss(output, target)
        # do a backward propagation 
        loss.backward()
        # update the weights
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    """
    Evalautes the model
    """
    # set the model to evaluation mode
    model.eval()
    # initialize the loss
    test_loss = 0
    correct = 0
    with torch.no_grad(): # deactivate autograd as we do not need a backprop here in eval
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Train the model

In [6]:
# This cell contains the actual train and evaluation calls
# Intialize, train and evaluate the model
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0770, Accuracy: 9780/10000 (97.80%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0446, Accuracy: 9871/10000 (98.71%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0234, Accuracy: 9921/10000 (99.21%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0204, Accuracy: 9926/10000 (99.26%)



loss=0.011274660937488079 batch_id=468: 100%|██████████| 469/469 [00:10<00:00, 45.75it/s]



Test set: Average loss: 0.0177, Accuracy: 9942/10000 (99.42%)



Achieved an accuracy of 99.42